##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Maker를 이용한 오디오 도메인 전이 학습

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/audio_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/lite/models/modify/model_maker/audio_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/lite/models/modify/model_maker/audio_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/lite/models/modify/model_maker/audio_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
  <td><a href="https://tfhub.dev/google/yamnet/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TF 허브 모델 보기</a></td>
</table>

이 colab 노트북에서는 [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker)를 사용하여 맞춤 오디오 분류 모델을 학습하는 방법을 배웁니다.

Model Maker 라이브러리는 전이 학습을 사용하여 사용자 지정 데이터세트를 사용하여 TensorFlow Lite 모델을 훈련하는 프로세스를 단순화합니다. 고유한 사용자 지정 데이터세트로 TensorFlow Lite 모델을 재훈련하면 필요한 훈련 데이터의 양과 시간이 줄어듭니다.

본 내용은 [오디오 모델을 사용자 지정하고 Android에 배포하기 위한 Codelab](https://codelabs.developers.google.com/codelabs/tflite-audio-classification-custom-model-android)의 일부입니다.

사용자 지정 새 데이터세트를 사용하고 전화기에서 사용할 수 있는 TFLite 모델, 브라우저에서 추론에 사용할 수 있는 TensorFlow.JS 모델 및 제공에 사용할 수 있는 SavedModel 버전을 내보냅니다.


## 종속성 설치하기


In [ ]:
!sudo apt -y install libportaudio2
!pip install tflite-model-maker

## TensorFlow, Model Maker 및 기타 라이브러리 가져오기

필요한 종속성 중에서 TensorFlow 및 Model Maker를 사용합니다. 그 외에 다른 것들은 오디오 조작, 재생 및 시각화를 위한 것입니다.

In [ ]:
import tensorflow as tf
import tflite_model_maker as mm
from tflite_model_maker import audio_classifier
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import glob
import random

from IPython.display import Audio, Image
from scipy.io import wavfile

print(f"TensorFlow Version: {tf.__version__}")
print(f"Model Maker Version: {mm.__version__}")

## 새 데이터세트

새 데이터세트는 5가지 유형의 새 노래로 구성된 교육 컬렉션입니다.

- 흰가슴 우드렌
- 참새
- 솔잣새
- 밤왕관 안피타
- 아자라 가시꼬리

원본 오디오는 전 세계의 새 소리를 공유하는 전용 웹사이트인 [Xeno-canto](https://www.xeno-canto.org/)에서 가져왔습니다.

데이터 다운로드부터 시작하겠습니다.

In [ ]:
birds_dataset_folder = tf.keras.utils.get_file('birds_dataset.zip',
                                                'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/birds_dataset.zip',
                                                cache_dir='./',
                                                cache_subdir='dataset',
                                                extract=True)
                                                

## 데이터 살펴보기

오디오는 이미 train 및 test 폴더로 분할되어 있습니다. 각 분할 폴더 안에는 `bird_code`를 이름으로 사용하는 각 새에 대해 하나의 폴더가 있습니다.

오디오는 모두 모노이며 16kHz 샘플 레이트입니다.

각 파일에 대한 자세한 내용은 `metadata.csv` 파일을 참조하세요. 여기에는 모든 파일 작성자, 라이선스 및 추가 정보가 포함되어 있습니다. 이 튜토리얼에서 직접 읽을 필요는 없습니다.

In [ ]:
# @title [Run this] Util functions and data structures.

data_dir = './dataset/small_birds_dataset'

bird_code_to_name = {
  'wbwwre1': 'White-breasted Wood-Wren',
  'houspa': 'House Sparrow',
  'redcro': 'Red Crossbill',  
  'chcant2': 'Chestnut-crowned Antpitta',
  'azaspi1': "Azara's Spinetail",   
}

birds_images = {
  'wbwwre1': 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/22/Henicorhina_leucosticta_%28Cucarachero_pechiblanco%29_-_Juvenil_%2814037225664%29.jpg/640px-Henicorhina_leucosticta_%28Cucarachero_pechiblanco%29_-_Juvenil_%2814037225664%29.jpg', # 	Alejandro Bayer Tamayo from Armenia, Colombia 
  'houspa': 'https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/House_Sparrow%2C_England_-_May_09.jpg/571px-House_Sparrow%2C_England_-_May_09.jpg', # 	Diliff
  'redcro': 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/49/Red_Crossbills_%28Male%29.jpg/640px-Red_Crossbills_%28Male%29.jpg', #  Elaine R. Wilson, www.naturespicsonline.com
  'chcant2': 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/67/Chestnut-crowned_antpitta_%2846933264335%29.jpg/640px-Chestnut-crowned_antpitta_%2846933264335%29.jpg', # 	Mike's Birds from Riverside, CA, US
  'azaspi1': 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/b2/Synallaxis_azarae_76608368.jpg/640px-Synallaxis_azarae_76608368.jpg', # https://www.inaturalist.org/photos/76608368
}

test_files = os.path.abspath(os.path.join(data_dir, 'test/*/*.wav'))

def get_random_audio_file():
  test_list = glob.glob(test_files)
  random_audio_path = random.choice(test_list)
  return random_audio_path


def show_bird_data(audio_path):
  sample_rate, audio_data = wavfile.read(audio_path, 'rb')

  bird_code = audio_path.split('/')[-2]
  print(f'Bird name: {bird_code_to_name[bird_code]}')
  print(f'Bird code: {bird_code}')
  display(Image(birds_images[bird_code]))

  plttitle = f'{bird_code_to_name[bird_code]} ({bird_code})'
  plt.title(plttitle)
  plt.plot(audio_data)
  display(Audio(audio_data, rate=sample_rate))

print('functions and data structures created')

### 일부 오디오 재생하기

데이터를 더 잘 이해하기 위해 테스트 분할에서 임의의 오디오 파일을 들어보겠습니다.

참고: 이 노트북의 뒷부분에서 테스트를 위해 이 오디오에 대한 추론을 실행할 것입니다.

In [ ]:
random_audio = get_random_audio_file()
show_bird_data(random_audio)

## 모델 훈련하기

오디오용 Model Maker를 사용할 때는 모델 사양으로 시작해야 합니다. 이것은 새 모델이 새 클래스에 대해 학습하기 위해 정보를 추출하는 기본 모델입니다. 또한 이는 샘플 레이트, 채널 수와 같은 모델 사양 매개변수를 준수하도록 데이터세트가 변환되는 방식에도 영향을 줍니다.

[YAMNet](https://tfhub.dev/google/yamnet/1)은 AudioSet 온톨로지에서 오디오 이벤트를 예측하기 위해 AudioSet 데이터세트에서 훈련된 오디오 이벤트 분류기입니다.

입력은 16kHz 및 1채널로 예상됩니다.

Model Maker가 알아서 처리하므로 직접 리샘플링을 수행할 필요가 없습니다.

- `frame_length`는 각 훈련 샘플의 길이를 결정합니다. 이 경우 EXPECTED_WAVEFORM_LENGTH * 3초입니다.

- `frame_steps`는 훈련 샘플이 얼마나 멀리 떨어져 있는지 결정합니다. 이 경우 i번째 샘플은 (i-1)번째 샘플 이후 EXPECTED_WAVEFORM_LENGTH * 6초에서 시작됩니다.

이러한 값을 설정하는 이유는 실제 데이터세트의 몇 가지 제한을 해결하기 위한 것입니다.

예를 들어, 새 데이터세트에서 새는 항상 노래하지 않습니다. 새는 노래하고, 쉬고, 노래하고, 그 사이에 소음도 있습니다. 프레임이 길면 노래를 캡처하는 데 도움이 되지만 너무 길게 설정하면 훈련용 샘플 수가 줄어듭니다.


In [ ]:
spec = audio_classifier.YamNetSpec(
    keep_yamnet_and_custom_heads=True,
    frame_step=3 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH,
    frame_length=6 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH)

## 데이터 로드하기

Model Maker에는 폴더에서 데이터를 로드하고 모델 사양에 대해 예상되는 형식으로 가져오는 API가 있습니다.

훈련 및 테스트 분할은 폴더를 기반으로 합니다. 유효성 검사 데이터세트는 훈련 분할의 20%로 생성됩니다.

참고: `cache=True`는 나중에 훈련을 더 빠르게 하는 데 중요하지만 데이터를 저장하려면 더 많은 RAM이 필요합니다. 새 데이터세트의 경우 300MB에 불과하기 때문에 문제가 되지 않지만, 자신의 데이터를 사용하는 경우에는 주의를 기울여야 합니다.


In [ ]:
train_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, 'train'), cache=True)
train_data, validation_data = train_data.split(0.8)
test_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, 'test'), cache=True)

## 모델 훈련하기

audio_classifier에는 모델을 생성하고 이미 훈련을 시작하는 [`create`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/create) 메서드가 있습니다.

많은 매개변수를 사용자 지정할 수 있습니다. 자세한 내용은 설명서를 참조하세요.

이 첫 번째 시도에서는 모든 기본 구성을 사용하고 100 epoch에 대해 학습합니다.

참고: 첫 번째 epoch는 캐시가 생성될 때이기 때문에 다른 모든 epoch보다 시간이 오래 걸립니다. 그 후 각 epoch는 1초에 가깝습니다.

In [ ]:
batch_size = 128
epochs = 100

print('Training the model')
model = audio_classifier.create(
    train_data,
    spec,
    validation_data,
    batch_size=batch_size,
    epochs=epochs)

정확도는 좋아 보이지만 테스트 데이터에 대해 평가 단계를 실행하고 모델이 시드되지 않은 데이터에서 좋은 결과를 얻었는지 확인하는 것이 중요합니다.

In [ ]:
print('Evaluating the model')
model.evaluate(test_data)

## 모델 이해하기

분류자를 훈련할 때 [혼동 행렬](https://en.wikipedia.org/wiki/Confusion_matrix)을 확인하는 것이 유용합니다. 혼동 행렬을 통해 분류자가 테스트 데이터에서 어떤 성능을 나타내는지 자세히 알아볼 수 있습니다.

Model Maker는 이미 혼동 행렬을 생성합니다.

In [ ]:
def show_confusion_matrix(confusion, test_labels):
  """Compute confusion matrix and normalize."""
  confusion_normalized = confusion.astype("float") / confusion.sum(axis=1)
  axis_labels = test_labels
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.2f', square=True)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

confusion_matrix = model.confusion_matrix(test_data)
show_confusion_matrix(confusion_matrix.numpy(), test_data.index_to_label)

## 모델 테스트하기 [선택 사항]

결과를 보기 위해 테스트 데이터세트의 샘플 오디오에 대한 모델을 시도할 수 있습니다.

먼저 제공 모델을 얻습니다.

In [ ]:
serving_model = model.create_serving_model()

print(f'Model\'s input shape and type: {serving_model.inputs}')
print(f'Model\'s output shape and type: {serving_model.outputs}')

이전에 로드한 임의의 오디오로 돌아갑니다.

In [ ]:
# if you want to try another file just uncoment the line below
random_audio = get_random_audio_file()
show_bird_data(random_audio)

생성된 모델에는 고정 입력 창이 있습니다.

주어진 오디오 파일의 경우, 예상 크기의 데이터 창으로 분할해야 합니다. 마지막 창은 0으로 채워야 할 수도 있습니다.

In [ ]:
sample_rate, audio_data = wavfile.read(random_audio, 'rb')

audio_data = np.array(audio_data) / tf.int16.max
input_size = serving_model.input_shape[1]

splitted_audio_data = tf.signal.frame(audio_data, input_size, input_size, pad_end=True, pad_value=0)

print(f'Test audio path: {random_audio}')
print(f'Original size of the audio data: {len(audio_data)}')
print(f'Number of windows for inference: {len(splitted_audio_data)}')

분할된 모든 오디오를 반복하고 각각에 대해 모델을 적용합니다.

방금 훈련한 모델에는 원본 YAMNet의 출력과 방금 훈련한 출력의 2가지 출력이 있습니다. 이는 실제 환경이 새 소리 이상으로 더 복잡하기 때문에 중요합니다. 예를 들어 새 사용 사례에서 YAMNet의 출력을 사용하여 관련 없는 오디오를 필터링할 수 있습니다. YAMNet이 새 또는 동물을 분류하지 않는 경우 모델의 출력이 관련 없는 분류를 가질 수 있음을 나타낼 수 있습니다.

관계를 쉽게 이해할 수 있도록 아래에 두 출력이 프린트되어 있습니다. 모델에서 저지르는 대부분의 실수는 YAMNet의 예측이 도메인(예: 새)과 관련이 없을 때입니다.

In [ ]:
print(random_audio)

results = []
print('Result of the window ith:  your model class -> score,  (spec class -> score)')
for i, data in enumerate(splitted_audio_data):
  yamnet_output, inference = serving_model(data)
  results.append(inference[0].numpy())
  result_index = tf.argmax(inference[0])
  spec_result_index = tf.argmax(yamnet_output[0])
  t = spec._yamnet_labels()[spec_result_index]
  result_str = f'Result of the window {i}: ' \
  f'\t{test_data.index_to_label[result_index]} -> {inference[0][result_index].numpy():.3f}, ' \
  f'\t({spec._yamnet_labels()[spec_result_index]} -> {yamnet_output[0][spec_result_index]:.3f})'
  print(result_str)


results_np = np.array(results)
mean_results = results_np.mean(axis=0)
result_index = mean_results.argmax()
print(f'Mean result: {test_data.index_to_label[result_index]} -> {mean_results[result_index]}')

## 모델 내보내기

마지막 단계는 임베디드 장치 또는 브라우저에서 사용할 모델을 내보내는 것입니다.

`export` 메서드는 두 형식을 모두 내보냅니다.

In [ ]:
models_path = './birds_models'
print(f'Exporing the TFLite model to {models_path}')

model.export(models_path, tflite_filename='my_birds_model.tflite')

Python 환경에서 제공하거나 사용하기 위해 SavedModel 버전을 내보낼 수도 있습니다.

In [ ]:
model.export(models_path, export_format=[mm.ExportFormat.SAVED_MODEL, mm.ExportFormat.LABEL])

## 다음 단계

잘 했습니다.

이제 [TFLite AudioClassifier Task API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/audio_classifier)를 사용하여 새 모델을 모바일 장치에 배포할 수 있습니다.

다른 클래스를 사용하여 자신의 데이터로 동일한 프로세스를 시도할 수도 있습니다. [오디오 분류를 위한 Model Maker](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier) 문서를 참조하세요.

또한 엔드 투 엔드 참조 앱인 [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/android/), [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/ios)을 통해 배워보세요.